In [1]:
# Import necessary libraries
import os
import glob
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report

In [2]:
#assuming we are in Q2 directory , we go to the parent folder
folder_path = os.path.join('..', 'new_data')

# Get all the CSV files from the folder
csv_files = glob.glob(folder_path + '/*.csv')

all_data=[]
# Load each CSV file in one dataframe
for idx,csv_file in enumerate(csv_files):
    # Load the CSV file into a dataframe
    df = pd.read_csv(csv_file)
    
    # Append the DataFrame to the list
    all_data.append(df)

# Combine all data into a single DataFrame
combined_data = pd.concat(all_data)


In [3]:
# Select relevant features and target variable
features = combined_data[['back_x', 'back_y', 'back_z', 'thigh_x', 'thigh_y', 'thigh_z']]
target = combined_data['label']

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.2, random_state=42)
 
# Create a Random Forest Classifier
rf_classifier = RandomForestClassifier(n_estimators=100, random_state=42)
 
# Train the classifier
rf_classifier.fit(X_train, y_train)
 
# Make predictions on the test set
y_pred = rf_classifier.predict(X_test)
 
# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
classification_rep = classification_report(y_test, y_pred)
 
# Print the results
print(f"Accuracy: {accuracy:.2f}")
print("\nClassification Report:\n", classification_rep)

Accuracy: 0.89

Classification Report:
               precision    recall  f1-score   support

           0       0.76      0.89      0.82     24198
           1       0.91      0.87      0.89      5745
           2       0.46      0.23      0.31      5074
           3       0.55      0.11      0.19      1540
           4       0.56      0.03      0.06      1385
           5       0.81      0.88      0.85     14805
           6       1.00      1.00      1.00     58003
           7       1.00      1.00      1.00      8523
           8       0.80      0.87      0.83      7838
           9       0.68      0.51      0.59      1104
          10       0.65      0.43      0.52       865
          11       0.69      0.42      0.53       149

    accuracy                           0.89    129229
   macro avg       0.74      0.60      0.63    129229
weighted avg       0.88      0.89      0.88    129229

